# Object Retrieval API
https://v2.sherpa.ac.uk/api/object-retrieval.html

In [35]:
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ['api_key']
# print(api_key)

In [87]:
import requests
import json
import pprint
# https://v2.sherpa.ac.uk/cgi/retrieve?item-type=repository&api-key=<YOUR-API-KEY>&format=Json&filter=[[%22country%22,%22equals%22,%22jp%22]]
url = 'https://v2.sherpa.ac.uk/cgi/retrieve'
filterparam = '[[%22country%22,%22equals%22,%22jp%22]]'
# python requests クエリストリングを encodeしないようにする
params = {'item-type':'repository','api-key':api_key,'format':'Json','filter':filterparam, 'offset':'600'}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
res = requests.get(url, params=params_str)
# print(res.url)
data = json.loads(res.text)
pprint.pprint(data)


{'items': [{'organisation': {'country': 'jp',
                             'country_phrases': [{'language': 'en',
                                                  'phrase': 'Japan',
                                                  'value': 'jp'}],
                             'name': [{'language': 'en',
                                       'language_phrases': [{'language': 'en',
                                                             'phrase': 'English',
                                                             'value': 'en'}],
                                       'name': 'Momoyamagakuin University',
                                       'preferred': 'name',
                                       'preferred_phrases': [{'language': 'en',
                                                              'phrase': 'Name',
                                                              'value': 'name'}]},
                                      {'language': 'ja',
                   

In [79]:
#import json 
#json_object = json.load(data)
#print(json_object["items"][0])
# すでに一つ上で json.load しているので、さらにloadしなくてよい（当たり前ですが・・・）
# data["items"][1] の　[0],[1],... がそれぞれ1機関
content = data["items"][99]
print(content)

{'organisation': {'country_phrases': [{'language': 'en', 'value': 'jp', 'phrase': 'Japan'}], 'country': 'jp', 'url': 'http://www.musashino-u.ac.jp/', 'name': [{'language': 'en', 'name': 'Musashino University', 'preferred_phrases': [{'phrase': 'Name', 'value': 'name', 'language': 'en'}], 'language_phrases': [{'phrase': 'English', 'value': 'en', 'language': 'en'}], 'preferred': 'name'}, {'language_phrases': [{'language': 'en', 'value': 'ja', 'phrase': 'Japanese'}], 'preferred': 'name', 'name': '武蔵野大学', 'preferred_phrases': [{'value': 'name', 'phrase': 'Name', 'language': 'en'}], 'language': 'ja'}]}, 'system_metadata': {'publicly_visible_phrases': [{'language': 'en', 'phrase': 'Yes', 'value': 'yes'}], 'publicly_visible': 'yes', 'uri': 'https://v2.sherpa.ac.uk/id/repository/3777', 'id': 3777, 'date_modified': '2022-01-12 15:35:53', 'date_created': '2016-11-10 15:08:49'}, 'repository_metadata': {'type_phrases': [{'language': 'en', 'value': 'institutional', 'phrase': 'Institutional'}], 'cont

In [80]:
len(data['items'])

100

In [81]:
def flatten(d,sep="_"):
    import collections
    obj = collections.OrderedDict()
    def recurse(t,parent_key=""):
        if isinstance(t,list):
            for i in range(len(t)):
                recurse(t[i],parent_key + sep + str(i) if parent_key else str(i))
        elif isinstance(t,dict):
            for k,v in t.items():
                recurse(v,parent_key + sep + k if parent_key else k)
        else:
            obj[parent_key] = t
    recurse(d)
    return obj

# data2 = flatten(data['items'])

for org in range(len(data['items'])):
    data2[org] = flatten(data['items'][org])  

In [82]:
# print(data2.keys())
data2

OrderedDict([('organisation_url', 'http://www.hokkyodai.ac.jp/'),
             ('organisation_country', 'jp'),
             ('organisation_country_phrases_0_phrase', 'Japan'),
             ('organisation_country_phrases_0_value', 'jp'),
             ('organisation_country_phrases_0_language', 'en'),
             ('organisation_name_0_language', 'en'),
             ('organisation_name_0_language_phrases_0_value', 'en'),
             ('organisation_name_0_language_phrases_0_phrase', 'English'),
             ('organisation_name_0_language_phrases_0_language', 'en'),
             ('organisation_name_0_preferred', 'name'),
             ('organisation_name_0_name', 'Hokkaido University of Education'),
             ('organisation_name_0_preferred_phrases_0_language', 'en'),
             ('organisation_name_0_preferred_phrases_0_phrase', 'Name'),
             ('organisation_name_0_preferred_phrases_0_value', 'name'),
             ('organisation_name_1_preferred_phrases_0_language', 'en'),
    

In [83]:
# Pandas Dataframe でキーを列に、値を行の値に変換するには、辞書を角括弧で囲むだけ
# 完全に一列フラットになってしまう
import pandas as pd
# repometa3 = pd.DataFrame([data2[0]])
# len(data2)
repometa3 = pd.DataFrame(index=[], columns=[])
for org in range(len(data['items'])):
    repometa3 = pd.concat([repometa3, pd.DataFrame([data2[org]])])
repometa3

,system_metadata_date_modified,system_metadata_date_created,system_metadata_id,system_metadata_uri,system_metadata_publicly_visible,system_metadata_publicly_visible_phrases_0_value,system_metadata_publicly_visible_phrases_0_phrase,system_metadata_publicly_visible_phrases_0_language,repository_metadata_metadata_record_count,repository_metadata_type_phrases_0_phrase,...,repository_metadata_policy_urls_0_type_phrases_0_phrase,repository_metadata_policy_urls_0_policy_url,repository_metadata_content_types_phrases_5_value,repository_metadata_content_types_phrases_5_phrase,repository_metadata_content_types_phrases_5_language,repository_metadata_content_types_phrases_6_value,repository_metadata_content_types_phrases_6_phrase,repository_metadata_content_types_phrases_6_language,repository_metadata_content_types_5,repository_metadata_content_types_6
0,2022-01-12 15:35:24,2010-10-20 10:10:51,1938,https://v2.sherpa.ac.uk/id/repository/1938,yes,yes,Yes,en,3176.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-01-12 15:35:24,2010-10-20 10:10:55,1939,https://v2.sherpa.ac.uk/id/repository/1939,yes,yes,Yes,en,15815.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-01-12 15:35:24,2010-10-20 11:11:01,1940,https://v2.sherpa.ac.uk/id/repository/1940,yes,yes,Yes,en,3205.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-01-12 15:35:24,2010-10-20 11:11:06,1941,https://v2.sherpa.ac.uk/id/repository/1941,yes,yes,Yes,en,10485.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-01-12 15:35:24,2010-10-20 11:11:11,1942,https://v2.sherpa.ac.uk/id/repository/1942,yes,yes,Yes,en,3818.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022-01-12 15:35:53,2016-11-10 14:24:21,3772,https://v2.sherpa.ac.uk/id/repository/3772,yes,yes,Yes,en,4139.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-01-12 15:35:53,2016-11-10 14:25:43,3773,https://v2.sherpa.ac.uk/id/repository/3773,yes,yes,Yes,en,646.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-01-12 15:35:53,2016-11-10 14:26:53,3774,https://v2.sherpa.ac.uk/id/repository/3774,yes,yes,Yes,en,1246.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-01-12 15:35:53,2016-11-10 14:28:05,3775,https://v2.sherpa.ac.uk/id/repository/3775,yes,yes,Yes,en,66.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
repometa3.set_index('system_metadata_id', drop=False, inplace=True)

In [85]:
repometa3
# repometa3.columns
# repometa3['organisation_name_1_name']
# print(repometa3.iloc[97:,])

,system_metadata_date_modified,system_metadata_date_created,system_metadata_id,system_metadata_uri,system_metadata_publicly_visible,system_metadata_publicly_visible_phrases_0_value,system_metadata_publicly_visible_phrases_0_phrase,system_metadata_publicly_visible_phrases_0_language,repository_metadata_metadata_record_count,repository_metadata_type_phrases_0_phrase,...,repository_metadata_policy_urls_0_type_phrases_0_phrase,repository_metadata_policy_urls_0_policy_url,repository_metadata_content_types_phrases_5_value,repository_metadata_content_types_phrases_5_phrase,repository_metadata_content_types_phrases_5_language,repository_metadata_content_types_phrases_6_value,repository_metadata_content_types_phrases_6_phrase,repository_metadata_content_types_phrases_6_language,repository_metadata_content_types_5,repository_metadata_content_types_6
system_metadata_id,,,,,,,,,,,,,,,,,,,,,
1938,2022-01-12 15:35:24,2010-10-20 10:10:51,1938,https://v2.sherpa.ac.uk/id/repository/1938,yes,yes,Yes,en,3176.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1939,2022-01-12 15:35:24,2010-10-20 10:10:55,1939,https://v2.sherpa.ac.uk/id/repository/1939,yes,yes,Yes,en,15815.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1940,2022-01-12 15:35:24,2010-10-20 11:11:01,1940,https://v2.sherpa.ac.uk/id/repository/1940,yes,yes,Yes,en,3205.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1941,2022-01-12 15:35:24,2010-10-20 11:11:06,1941,https://v2.sherpa.ac.uk/id/repository/1941,yes,yes,Yes,en,10485.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1942,2022-01-12 15:35:24,2010-10-20 11:11:11,1942,https://v2.sherpa.ac.uk/id/repository/1942,yes,yes,Yes,en,3818.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,2022-01-12 15:35:53,2016-11-10 14:24:21,3772,https://v2.sherpa.ac.uk/id/repository/3772,yes,yes,Yes,en,4139.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3773,2022-01-12 15:35:53,2016-11-10 14:25:43,3773,https://v2.sherpa.ac.uk/id/repository/3773,yes,yes,Yes,en,646.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3774,2022-01-12 15:35:53,2016-11-10 14:26:53,3774,https://v2.sherpa.ac.uk/id/repository/3774,yes,yes,Yes,en,1246.0,Institutional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
import pandas as pd
repometa = pd.DataFrame(list(content.items())).T
# repometa = pd.DataFrame.from_dict(content, orient='index').T
repometa

,0,1,2
0,organisation,system_metadata,repository_metadata
1,{'identifiers': [{'type_phrases': [{'value': '...,"{'publicly_visible_phrases': [{'value': 'yes',...",{'software': {'name_phrases': [{'phrase': 'WEK...


In [7]:
# content2 = data["items"]
# print(content2)
repometa2 = pd.DataFrame(data["items"])
repometa2

,organisation,system_metadata,repository_metadata
0,"{'country_phrases': [{'value': 'jp', 'language...","{'publicly_visible': 'yes', 'uri': 'https://v2...",{'url': 'https://opac.ll.chiba-u.jp/da/curator...
1,{'identifiers': [{'type_phrases': [{'value': '...,"{'publicly_visible_phrases': [{'value': 'yes',...",{'software': {'name_phrases': [{'phrase': 'WEK...
2,"{'url': 'https://www.hokudai.ac.jp', 'name': [...",{'publicly_visible_phrases': [{'phrase': 'Yes'...,"{'name': [{'preferred': 'name', 'language_phra..."
3,{'name': [{'preferred_phrases': [{'phrase': 'N...,"{'date_modified': '2022-01-12 15:35:00', 'publ...","{'content_types': ['journal_articles', 'confer..."
4,"{'url': 'http://www.kyushu-u.ac.jp/', 'name': ...","{'date_modified': '2022-01-12 15:35:00', 'uri'...",{'url': 'https://www.lib.kyushu-u.ac.jp/ja/col...
...,...,...,...
95,"{'country': 'jp', 'country_phrases': [{'phrase...",{'publicly_visible_phrases': [{'language': 'en...,"{'content_types': ['journal_articles', 'theses..."
96,{'identifiers': [{'identifier': 'https://ror.o...,"{'date_created': '2010-10-13 14:14:34', 'id': ...",{'software': {'name_phrases': [{'phrase': 'Xoo...
97,"{'country_phrases': [{'phrase': 'Japan', 'valu...","{'date_created': '2010-10-20 10:10:10', 'id': ...","{'content_subjects': ['science', 'technology',..."
98,"{'identifiers': [{'type': 'ror', 'identifier':...","{'publicly_visible': 'yes', 'uri': 'https://v2...","{'name': [{'preferred': 'acronym', 'language_p..."


In [ ]:
repometa2

In [8]:
repometa3 = repometa2.copy()

In [13]:
# pandasのセルのなかにdictionaryが！を展開
# repometa4 = repometa2['repository_metadata'].apply(pd.Series)
# これだけだとさらにdarrayが埋まっている
'''
for i, column_name in enumerate(repometa3):
   repometa4 = repometa3[column_name].apply(pd.Series)
'''

def flatdict():
   for i, column_name in enumerate(repometa3):
      repometa4 = repometa3[column_name].apply(pd.Series)
   return repometa4

repometa4 = flatdict()


In [16]:
# data['items'][0]
repometa4

,url,oai_url,content_types_phrases,type,content_types,full_text_record_count,content_subjects_phrases,name,content_subjects,software,type_phrases,metadata_record_count,notes,policy_urls
0,https://opac.ll.chiba-u.jp/da/curator/?lang=1,https://opac.ll.chiba-u.jp/mmd_api/oai-pmh/,"[{'value': 'journal_articles', 'language': 'en...",institutional,"[journal_articles, conference_and_workshop_pap...",42971.0,"[{'phrase': 'Science', 'language': 'en', 'valu...","[{'language_phrases': [{'phrase': 'English', '...","[science, technology, engineering, mathematics...","{'name': 'other', 'name_other': 'E-Cats Librar...","[{'phrase': 'Institutional', 'value': 'institu...",97628,NaN,NaN
1,https://waseda.repo.nii.ac.jp/,http://waseda.repo.nii.ac.jp/oai,"[{'phrase': 'Journal Articles', 'value': 'jour...",institutional,"[journal_articles, conference_and_workshop_pap...",39513.0,"[{'value': 'science', 'language': 'en', 'phras...","[{'preferred': 'name', 'language_phrases': [{'...","[science, technology, engineering, mathematics...","{'name_phrases': [{'phrase': 'WEKO', 'value': ...","[{'value': 'institutional', 'language': 'en', ...",46388,NaN,NaN
2,https://eprints.lib.hokudai.ac.jp/,http://eprints.lib.hokudai.ac.jp/dspace-oai/re...,"[{'language': 'en', 'value': 'journal_articles...",institutional,"[journal_articles, bibliographic_references, c...",NaN,"[{'phrase': 'Science', 'value': 'science', 'la...","[{'preferred': 'name', 'language_phrases': [{'...","[science, technology, engineering, mathematics...","{'name_phrases': [{'value': 'dspace', 'languag...","[{'value': 'institutional', 'language': 'en', ...",57986,NaN,NaN
3,https://ir.lib.hiroshima-u.ac.jp/ja,http://ir.lib.hiroshima-u.ac.jp/api/oai/request,"[{'phrase': 'Journal Articles', 'value': 'jour...",institutional,"[journal_articles, conference_and_workshop_pap...",NaN,"[{'language': 'en', 'value': 'science', 'phras...","[{'acronym': 'HiR', 'name': 'Hiroshima Univers...","[science, technology, engineering, mathematics...","{'name': 'other', 'name_other': 'E-Repository'...","[{'phrase': 'Institutional', 'language': 'en',...",27136,NaN,NaN
4,https://www.lib.kyushu-u.ac.jp/ja/collections/qir,http://catalog.lib.kyushu-u.ac.jp/mmd/mmd_api/...,"[{'value': 'journal_articles', 'language': 'en...",institutional,"[journal_articles, theses_and_dissertations, u...",NaN,"[{'phrase': 'Science', 'value': 'science', 'la...","[{'preferred': 'name', 'name': 'Kyushu Univers...","[science, technology, engineering, mathematics...","{'name': 'other', 'name_other': 'E-Cats Librar...","[{'language': 'en', 'value': 'institutional', ...",79436,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://library.bliss.chubu.ac.jp/?action=page...,http://elib.bliss.chubu.ac.jp/oai/oai-pmh.do,"[{'language': 'en', 'value': 'journal_articles...",institutional,"[journal_articles, theses_and_dissertations, u...",0.0,"[{'value': 'science', 'language': 'en', 'phras...","[{'name': 'Chubu University Digital Archives',...","[science, technology, engineering, mathematics...","{'name_phrases': [{'phrase': 'Other', 'value':...","[{'phrase': 'Institutional', 'value': 'institu...",3589,NaN,NaN
96,http://repo.beppu-u.ac.jp,http://repo.beppu-u.ac.jp/modules/xoonips/oai.php,"[{'language': 'en', 'value': 'journal_articles...",institutional,"[journal_articles, bibliographic_references]",NaN,"[{'language': 'en', 'value': 'science', 'phras...","[{'preferred_phrases': [{'value': 'name', 'lan...","[science, technology, engineering, mathematics...","{'name_phrases': [{'phrase': 'XooNIps', 'langu...","[{'phrase': 'Institutional', 'language': 'en',...",4983,NaN,NaN
97,http://repository.ri.jica.go.jp/dspace/,NaN,"[{'language': 'en', 'value': 'unpub_reports_an...",institutional,[unpub_reports_and_working_papers],NaN,"[{'language': 'en', 'value': 'science', 'phras...","[{'preferred': 'name', 'language': 'en', 'pref...","[science, technology, engineering, mathematics...","{'name': 'dspace', 'name_phrases': [{'phrase':...","[{'languag

In [26]:
#v for i, column_name in enumerate(repometa4):
#    print(column_name)
#for i, column_name in enumerate(repometa4):
column_name = 'content_types_phrases'
repometa5 = repometa4[column_name].apply(pd.Series)

In [27]:
repometa5

,0,1,2,3,4,5,6
0,"{'value': 'journal_articles', 'language': 'en'...","{'language': 'en', 'value': 'conference_and_wo...","{'language': 'en', 'value': 'theses_and_disser...","{'value': 'unpub_reports_and_working_papers', ...","{'language': 'en', 'value': 'books_chapters_an...","{'language': 'en', 'value': 'datasets', 'phras...","{'phrase': 'Learning Objects', 'language': 'en..."
1,"{'phrase': 'Journal Articles', 'value': 'journ...","{'phrase': 'Conference and Workshop Papers', '...","{'language': 'en', 'value': 'theses_and_disser...","{'language': 'en', 'value': 'unpub_reports_and...",NaN,NaN,NaN
2,"{'language': 'en', 'value': 'journal_articles'...","{'language': 'en', 'value': 'bibliographic_ref...","{'language': 'en', 'value': 'conference_and_wo...","{'language': 'en', 'value': 'theses_and_disser...","{'phrase': 'Reports and Working Papers', 'valu...","{'value': 'other_special_item_types', 'languag...",NaN
3,"{'phrase': 'Journal Articles', 'value': 'journ...","{'language': 'en', 'value': 'conference_and_wo...","{'phrase': 'Theses and Dissertations', 'langua...","{'phrase': 'Reports and Working Papers', 'lang...","{'phrase': 'Books, Chapters and Sections', 'la...","{'value': 'learning_objects', 'language': 'en'...","{'phrase': 'Other Special Item Types', 'value'..."
4,"{'value': 'journal_articles', 'language': 'en'...","{'phrase': 'Theses and Dissertations', 'langua...","{'value': 'unpub_reports_and_working_papers', ...","{'language': 'en', 'value': 'books_chapters_an...","{'phrase': 'Other Special Item Types', 'value'...",NaN,NaN
...,...,...,...,...,...,...,...
95,"{'language': 'en', 'value': 'journal_articles'...","{'phrase': 'Theses and Dissertations', 'langua...","{'phrase': 'Reports and Working Papers', 'lang...",NaN,NaN,NaN,NaN
96,"{'language': 'en', 'value': 'journal_articles'...","{'phrase': 'Bibliographic References', 'langua...",NaN,NaN,NaN,NaN,NaN
97,"{'language': 'en', 'value': 'unpub_reports_and...",NaN,NaN,NaN,NaN,NaN,NaN
98,"{'language': 'en', 'value': 'journal_articles'...","{'phrase': 'Conference and Workshop Papers', '...",NaN,NaN,NaN,NaN,NaN


In [20]:
def flatdict():
   for i, column_name in enumerate(repometa4):
      repometa5 = repometa4[column_name].apply(pd.Series)
   return repometa5
flatdict()

,0,1,2,3,4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN


In [21]:
repometa4

,url,oai_url,content_types_phrases,type,content_types,full_text_record_count,content_subjects_phrases,name,content_subjects,software,type_phrases,metadata_record_count,notes,policy_urls
0,https://opac.ll.chiba-u.jp/da/curator/?lang=1,https://opac.ll.chiba-u.jp/mmd_api/oai-pmh/,"[{'value': 'journal_articles', 'language': 'en...",institutional,"[journal_articles, conference_and_workshop_pap...",42971.0,"[{'phrase': 'Science', 'language': 'en', 'valu...","[{'language_phrases': [{'phrase': 'English', '...","[science, technology, engineering, mathematics...","{'name': 'other', 'name_other': 'E-Cats Librar...","[{'phrase': 'Institutional', 'value': 'institu...",97628,NaN,NaN
1,https://waseda.repo.nii.ac.jp/,http://waseda.repo.nii.ac.jp/oai,"[{'phrase': 'Journal Articles', 'value': 'jour...",institutional,"[journal_articles, conference_and_workshop_pap...",39513.0,"[{'value': 'science', 'language': 'en', 'phras...","[{'preferred': 'name', 'language_phrases': [{'...","[science, technology, engineering, mathematics...","{'name_phrases': [{'phrase': 'WEKO', 'value': ...","[{'value': 'institutional', 'language': 'en', ...",46388,NaN,NaN
2,https://eprints.lib.hokudai.ac.jp/,http://eprints.lib.hokudai.ac.jp/dspace-oai/re...,"[{'language': 'en', 'value': 'journal_articles...",institutional,"[journal_articles, bibliographic_references, c...",NaN,"[{'phrase': 'Science', 'value': 'science', 'la...","[{'preferred': 'name', 'language_phrases': [{'...","[science, technology, engineering, mathematics...","{'name_phrases': [{'value': 'dspace', 'languag...","[{'value': 'institutional', 'language': 'en', ...",57986,NaN,NaN
3,https://ir.lib.hiroshima-u.ac.jp/ja,http://ir.lib.hiroshima-u.ac.jp/api/oai/request,"[{'phrase': 'Journal Articles', 'value': 'jour...",institutional,"[journal_articles, conference_and_workshop_pap...",NaN,"[{'language': 'en', 'value': 'science', 'phras...","[{'acronym': 'HiR', 'name': 'Hiroshima Univers...","[science, technology, engineering, mathematics...","{'name': 'other', 'name_other': 'E-Repository'...","[{'phrase': 'Institutional', 'language': 'en',...",27136,NaN,NaN
4,https://www.lib.kyushu-u.ac.jp/ja/collections/qir,http://catalog.lib.kyushu-u.ac.jp/mmd/mmd_api/...,"[{'value': 'journal_articles', 'language': 'en...",institutional,"[journal_articles, theses_and_dissertations, u...",NaN,"[{'phrase': 'Science', 'value': 'science', 'la...","[{'preferred': 'name', 'name': 'Kyushu Univers...","[science, technology, engineering, mathematics...","{'name': 'other', 'name_other': 'E-Cats Librar...","[{'language': 'en', 'value': 'institutional', ...",79436,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://library.bliss.chubu.ac.jp/?action=page...,http://elib.bliss.chubu.ac.jp/oai/oai-pmh.do,"[{'language': 'en', 'value': 'journal_articles...",institutional,"[journal_articles, theses_and_dissertations, u...",0.0,"[{'value': 'science', 'language': 'en', 'phras...","[{'name': 'Chubu University Digital Archives',...","[science, technology, engineering, mathematics...","{'name_phrases': [{'phrase': 'Other', 'value':...","[{'phrase': 'Institutional', 'value': 'institu...",3589,NaN,NaN
96,http://repo.beppu-u.ac.jp,http://repo.beppu-u.ac.jp/modules/xoonips/oai.php,"[{'language': 'en', 'value': 'journal_articles...",institutional,"[journal_articles, bibliographic_references]",NaN,"[{'language': 'en', 'value': 'science', 'phras...","[{'preferred_phrases': [{'value': 'name', 'lan...","[science, technology, engineering, mathematics...","{'name_phrases': [{'phrase': 'XooNIps', 'langu...","[{'phrase': 'Institutional', 'language': 'en',...",4983,NaN,NaN
97,http://repository.ri.jica.go.jp/dspace/,NaN,"[{'language': 'en', 'value': 'unpub_reports_an...",institutional,[unpub_reports_and_working_papers],NaN,"[{'language': 'en', 'value': 'science', 'phras...","[{'preferred': 'name', 'language': 'en', 'pref...","[science, technology, engineering, mathematics...","{'name': 'dspace', 'name_phrases': [{'phrase':...","[{'languag

In [ ]:

# for i, column_name in enumerate(repometa3):
column_name = 'software'
df = pd.DataFrame(repometa4[column_name].values.tolist()).add_prefix(column_name)
# df = pd.DataFrame(repometa4['content_types'].values.tolist()).add_prefix('content_types_')
# df = pd.DataFrame(repometa2['type'].values.tolist()).add_prefix('content_types_')
'''
def flatarray():
   for i, column_name in enumerate(repometa2):
       pd.DataFrame(repometa2[column_name].values.tolist()).add_prefix(column_name)
       # df2 = pd.DataFrame(repometa2[columun_name].values.tolist()).add_prefix(column_name)
       # repometa3 = pd.concat(repometa2, df2)
       # repometa2.assign(df2)
       # repometa2[column_name].apply(pd.DataFrame(repometa2[columun_name].values.tolist()))
       # repometa2[column_name].apply(pd.Series)
   return flatarray()

df2 = flatarray()
'''



In [ ]:
df

In [ ]:
def _unwrap(dct, prefix):
    for key, value in dct.items():
        new_key = f'{prefix}_{key}'
        if isinstance(value, dict):
            yield from _unwrap(value, new_key)
        else:
            yield new_key[1:], value       

def unwrap(dct):
    return dict(_unwrap(dct, ''))

content2 = repometa2['name']
d2 = unwrap(content2)
d2


In [ ]:
repometa2 = pd.DataFrame(data["items"])
repometa2